In [144]:
import pandas as pd
import numpy as np

import nltk
# nltk.download()
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split
# !pip install -U gensim
import gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from keras.utils import to_categorical

In [145]:
data = pd.read_csv("Resources/CL_data/train.csv")

In [146]:
data.head()

,Id,Category,Tweet
0,639976673685962000,0,And @whytology article 3 gives SCOTUS no legis...
1,641539752177586000,0,And 2nd it wasnt harrys fault that he had ran ...
2,639767721073315000,0,And I don't know about TGIF. Fridays are terri...
3,638850695349800000,0,"And if my day wasn't bad enough already, Snoop..."
4,638783969568366000,0,And in comes Sony with the letdown. Only one c...


In [147]:
data.rename(columns={'Id':'id',
                          'Category':'label',
                          'Tweet':'tweet'}, 
                 inplace=True)

In [148]:
data.head()

,id,label,tweet
0,639976673685962000,0,And @whytology article 3 gives SCOTUS no legis...
1,641539752177586000,0,And 2nd it wasnt harrys fault that he had ran ...
2,639767721073315000,0,And I don't know about TGIF. Fridays are terri...
3,638850695349800000,0,"And if my day wasn't bad enough already, Snoop..."
4,638783969568366000,0,And in comes Sony with the letdown. Only one c...


In [149]:
data.shape

(2742, 3)

In [150]:
data["label"].value_counts()

1    2165
0     577
Name: label, dtype: int64

## Cleaning the data

In [151]:
# dropping id column

data = data.drop("id", axis=1)

In [152]:
stopwords = stopwords.words('english')

In [153]:
def tweet_cleaner(tweet):
    tweet = re.sub(r"@\w*", " ", str(tweet).lower()).strip() #removing username
    tweet = re.sub(r'https?://[A-Za-z0-9./]+', " ", str(tweet).lower()).strip() #removing links
    tweet = re.sub(r'[^a-zA-Z]', " ", str(tweet).lower()).strip() #removing sp_char
    tw = []
    
    for text in tweet.split():
        if text not in stopwords:
            tw.append(text)
    
    return " ".join(tw)

In [154]:
data.tweet = data.tweet.apply(lambda x: tweet_cleaner(x))

### word2vec

In [155]:
documents = [text.split() for text in data.tweet]

In [156]:
len(documents)

2742

In [157]:
w2v_model = gensim.models.word2vec.Word2Vec(size = 256, window = 7, min_count = 5)

In [158]:
w2v_model.build_vocab(documents)

In [159]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(593923, 967424)

In [160]:
w2v_model.wv["happy"]

array([ 0.06590464, -0.3196186 , -0.06217399,  0.14874361,  0.40737784,
       -0.13573837, -0.0475671 ,  0.06620029, -0.3383426 , -0.26824862,
        0.4216163 , -0.37650153, -0.4973726 ,  0.1038716 , -0.1974961 ,
        0.15772966,  0.23839957, -0.06445148, -0.20530394,  0.31666464,
        0.0016158 , -0.1569447 ,  0.60311073, -0.29148468, -0.1695796 ,
       -0.06789075, -0.0683419 ,  0.0537324 ,  0.05842369, -0.17793119,
        0.299001  , -0.13697407, -0.08911684, -0.2627426 , -0.10975121,
        0.0563928 ,  0.22179131,  0.05979598,  0.41608202, -0.38272715,
       -0.00415148, -0.12308431, -0.11639325, -0.2698516 ,  0.4435986 ,
        0.03127245, -0.28358015,  0.08813322, -0.00646646,  0.33259115,
        0.01340492, -0.00255785,  0.0115562 ,  0.16007955,  0.28575575,
        0.01579565, -0.28520483,  0.3184867 , -0.22819598, -0.5487315 ,
        0.2371987 ,  0.1259444 , -0.03294426,  0.24100785, -0.12397758,
        0.14174476,  0.02814276,  0.13918383,  0.03450953,  0.09

### Converting tweets to vectors

In [161]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.tweet)

In [162]:
len(tokenizer.word_index)

7011

In [163]:
x_train = pad_sequences(tokenizer.texts_to_sequences(data.tweet), maxlen=256, padding="post", truncating="post")

In [164]:
x_train

array([[1045, 1046,  363, ...,    0,    0,    0],
       [  28, 1779, 2769, ...,    0,    0,    0],
       [  33,   37, 2772, ...,    0,    0,    0],
       ...,
       [ 110,  220,    2, ...,    0,    0,    0],
       [ 110, 1262,    2, ...,    0,    0,    0],
       [ 110, 7009, 7010, ...,    0,    0,    0]], dtype=int32)

In [165]:
y_train = data.label

y_train_f = []
for x in y_train:
    if x == 1:
        y_train_f.append(1)
    elif x == 0:
        y_train_f.append(0)
    elif x == -1:
        y_train_f.append(2)
        
y_train_f = to_categorical(y_train_f)

In [166]:
y_train_f

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

### Model

In [167]:
embedding_matrix = np.zeros((14850,256))

In [168]:
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [169]:
embedding_layer = Embedding(14850, 256, weights=[embedding_matrix], input_length=256, trainable=False)

In [170]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.25))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(50, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [171]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 256)          3801600   
_________________________________________________________________
dropout_7 (Dropout)          (None, 256, 256)          0         
_________________________________________________________________
dense_10 (Dense)             (None, 256, 200)          51400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256, 200)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 102       
Total para

In [172]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [173]:
model.fit(x_train, y_train_f, batch_size=32, epochs=6, validation_split=0.1, verbose=1)

Train on 2467 samples, validate on 275 samples
Epoch 1/6
2467/2467 [==============================] - 32s 13ms/step - loss: 0.5632 - acc: 0.7580 - val_loss: 0.2960 - val_acc: 1.0000
Epoch 2/6
2467/2467 [==============================] - 29s 12ms/step - loss: 0.5490 - acc: 0.7661 - val_loss: 0.2230 - val_acc: 1.0000
Epoch 3/6
2467/2467 [==============================] - 30s 12ms/step - loss: 0.5474 - acc: 0.7661 - val_loss: 0.2318 - val_acc: 1.0000
Epoch 4/6
2467/2467 [==============================] - 32s 13ms/step - loss: 0.5470 - acc: 0.7661 - val_loss: 0.2273 - val_acc: 1.0000
Epoch 5/6
2467/2467 [==============================] - 33s 13ms/step - loss: 0.5476 - acc: 0.7661 - val_loss: 0.2858 - val_acc: 1.0000
Epoch 6/6
2467/2467 [==============================] - 31s 13ms/step - loss: 0.5433 - acc: 0.7661 - val_loss: 0.3419 - val_acc: 1.0000


# Testing

In [182]:
def sentiment(text):
    
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=256)
    score = model.predict([x_test])[0]
    
    final = "Positive = %f ,Negative = %f" % (score[1], score[0])
    return print(final)

In [186]:
sentiment("dont tell")

Positive = 0.739286 ,Negative = 0.260714
